In [6]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
%matplotlib inline

In [4]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.dpi']= 180

In [12]:
os.path.abspath(os.path.join(os.getcwd(), '..', 'job_ads.csv'))

'/Users/ruslan.kozhuharov/projects/analyses/ggl_job_ads/job_ads.csv'

In [13]:
file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'job_ads.csv'))
ads = pd.read_csv(file_path, index_col=0)
ads.drop_duplicates(inplace=True)

In [15]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 595 entries, 0 to 371
Data columns (total 7 columns):
id                  595 non-null object
job_descr           477 non-null object
location            591 non-null object
minimum_qual        477 non-null object
preferred_qual      477 non-null object
responsibilities    477 non-null object
title               595 non-null object
dtypes: object(7)
memory usage: 37.2+ KB
